# HistoricalPricing content objects - Event Driven

This script shows how to request Historical Pricing data in Event Driven mode  


## Import the Refinitifv Data Platform Library

In [ ]:
import refinitiv.dataplatform as rdp
import datetime
import asyncio
import pandas as pd

## Get credentials

In [ ]:
%run -i ../Sessions.ipynb

## Create a session and open it 

In [ ]:
mySession = get_session("platform")
mySession.open()

## Create the Historical Pricing object with a response callback

In [ ]:
def display_response(response):
    data_type = response.closure
    current_time = datetime.datetime.now().time()
    display('{} received at {}'.format(data_type, current_time))
    display(response.data.dataframe)

In [ ]:
historical_pricing = rdp.HistoricalPricing(
    session = mySession,
    on_response = lambda hp, response : display_response(response)
)

## Historical Price Events + Intraday Summaries + Interday Summaries

In [ ]:
my_adjustments = [
    rdp.Adjustments.EXCHANGE_CORRECTION, 
    rdp.Adjustments.MANUAL_CORRECTION,
    rdp.Adjustments.CCH,
    rdp.Adjustments.CRE,
    rdp.Adjustments.RPO,
    rdp.Adjustments.RTS,
]

historical_events = asyncio.get_event_loop().create_task(
    historical_pricing.get_events_async(
        universe = 'VOD.L', 
        count = 10,
        eventTypes = [rdp.EventTypes.TRADE], 
        closure = 'Historical Price Events')
)

intraday_summaries = asyncio.get_event_loop().create_task(
    historical_pricing.get_summaries_async(
        universe = 'VOD.L', 
        count = 6,
        interval = rdp.Intervals.TEN_MINUTES, 
        adjustments = my_adjustments, 
        closure = 'Historical Intraday Summaries')
)

interday_summaries = asyncio.get_event_loop().create_task(
    historical_pricing.get_summaries_async(
        universe = 'VOD.L', 
        count = 5,
        interval = rdp.Intervals.DAILY, 
        adjustments = my_adjustments, 
        closure = 'Historical Interday Summaries')
)

## Close the session

In [ ]:
mySession.close()